In [25]:
import numpy as np
import pandas as pd
import scipy
import implicit
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares
import os

In [17]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'count'}, inplace=True)

movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


## 분석해 봅시다.

In [18]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [19]:
# rating에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [20]:
# 가장 인기있는 영화 30개(인기순)
movie_data = pd.merge(ratings, movies)
movie_count = movie_data.groupby('title')['count'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

## 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.

In [22]:
my_favorite = ['Star Wars: Episode IV - A New Hope (1977)' , 'Star Wars: Episode V - The Empire Strikes Back (1980)' ,'Star Wars: Episode VI - Return of the Jedi (1983)' ,'Men in Black (1997)' ,'Toy Story (1995)']

favorite_movie_id = movies[movies['title'].isin(my_favorite)]
my_movie = pd.DataFrame({'user_id': [6041]*5, 'movie_id': favorite_movie_id['movie_id'], 'count':[5]*5})

if not ratings.isin({'user_id':[6041]})['user_id'].any():
    ratings = ratings.append(my_movie)

In [23]:
print(ratings)

      user_id  movie_id  count    timestamp
0           1      1193      5  978300760.0
1           1       661      3  978302109.0
2           1       914      3  978301968.0
3           1      3408      4  978300275.0
4           1      2355      5  978824291.0
...       ...       ...    ...          ...
0        6041         1      5          NaN
257      6041       260      5          NaN
1178     6041      1196      5          NaN
1192     6041      1210      5          NaN
1539     6041      1580      5          NaN

[836483 rows x 4 columns]


## Making CSR matrix

In [26]:
csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)))
csr_data

<6042x3953 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## Training

In [27]:
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)
csr_data_transpose = csr_data.T

In [28]:
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

## 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [35]:
print(movies.loc[movies['movie_id'] == 1196])

      movie_id                                              title  \
1178      1196  Star Wars: Episode V - The Empire Strikes Back...   

                                  genre  
1178  Action|Adventure|Drama|Sci-Fi|War  


In [30]:
my_vector, starwars5_vector = als_model.user_factors[6041], als_model.item_factors[1196]

In [31]:
print(np.dot(my_vector, starwars5_vector))

0.7916984


In [39]:
toystory_vector = als_model.item_factors[1]
print(np.dot(my_vector, toystory_vector))

0.42848623


## 비슷한 영화 추천

In [41]:
favorite_movie = 'Star Wars: Episode V - The Empire Strikes Back (1980)'
movie_id = movies[movies['title']=='Godfather, The (1972)']['movie_id']
similar_movie = als_model.similar_items(movie_id.values[0], N=15)
print(movies[movies['movie_id'].isin([s[0] for s in similar_movie])])

      movie_id                                              title  \
109        111                                 Taxi Driver (1976)   
619        624                               Condition Red (1995)   
847        858                              Godfather, The (1972)   
861        872                               Aiqing wansui (1994)   
911        923                                Citizen Kane (1941)   
1195      1213                                  GoodFellas (1990)   
1203      1221                     Godfather: Part II, The (1974)   
1232      1252                                   Chinatown (1974)   
1728      1787  Paralyzing Fear: The Story of Polio in America...   
1884      1953                      French Connection, The (1971)   
1954      2023                    Godfather: Part III, The (1990)   
2626      2695                                   Boys, The (1997)   
2818      2887                                   Simon Sez (1999)   
3293      3362                    

## 좋아할 만한 영화 추천

In [43]:
movie_recommended = als_model.recommend(6041, csr_data, N=20, filter_already_liked_items=True)
print(movie_recommended)

[(1198, 0.47771493), (2628, 0.4637355), (3114, 0.3921763), (480, 0.3747278), (356, 0.37019545), (589, 0.35938245), (1270, 0.35367382), (2571, 0.33695787), (2858, 0.33235317), (2355, 0.3218944), (1197, 0.31003478), (1097, 0.2758375), (3175, 0.25518152), (1240, 0.25488025), (34, 0.2518977), (1265, 0.24610902), (110, 0.20838612), (1291, 0.2083039), (588, 0.20472558), (780, 0.17783237)]


In [44]:
print(movies[movies['movie_id'].isin([m[0] for m in movie_recommended])])

      movie_id                                             title  \
33          34                                       Babe (1995)   
108        110                                 Braveheart (1995)   
352        356                               Forrest Gump (1994)   
476        480                              Jurassic Park (1993)   
584        588                                    Aladdin (1992)   
585        589                 Terminator 2: Judgment Day (1991)   
770        780                     Independence Day (ID4) (1996)   
1081      1097                 E.T. the Extra-Terrestrial (1982)   
1179      1197                        Princess Bride, The (1987)   
1180      1198                    Raiders of the Lost Ark (1981)   
1220      1240                            Terminator, The (1984)   
1245      1265                              Groundhog Day (1993)   
1250      1270                         Back to the Future (1985)   
1271      1291         Indiana Jones and the Las

## 회고
---
스타워즈를 3개나 골라, 스타워즈 작품과 dot한 결과가 0.79나 되었다. 토이스토리 벡터와 곱해보니 0.42정도가 나왔다. 토이스토리보다는 스타워즈 작품들에 선호가 더 크게 나왔다. 좋아할 만한 영화에 다른 작품은 잘 모르겠고, 스타워즈 1이 끼어있는 것을 보니, 제대로 된 결과가 나온 듯하다. 